<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
import gspread
import pandas as pd
import os
import numpy as np

from dotenv import load_dotenv
from google.oauth2 import service_account

# === Carregar variáveis de ambiente ===
load_dotenv()

# === Definir escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

# === Caminho seguro para o arquivo JSON (lido do .env) ===
cred_path = os.getenv('GOOGLE_CREDENTIALS_PATH')

# === Carregar credenciais de forma moderna ===
creds = service_account.Credentials.from_service_account_file(
    cred_path,
    scopes=scopes
)

# === Autorizar o cliente gspread ===
client = gspread.authorize(creds)

# === Função para carregar aba da planilha ===
def carregar_aba(sheet_id, aba_nome):
    planilha = client.open_by_key(sheet_id)
    aba = planilha.worksheet(aba_nome)
    dados = aba.get_all_records()
    return pd.DataFrame(dados)

# === IDs das planilhas (fixar no .env depois se quiser) ===
id_dados_leads_ssp = "1K7nLxfgu6ktdyFwlHMPJsj5xdv410tensfPccmmPDm8"
id_pesquisa_ssp = "1pK0BP8n98DtQ8OwGeBhzcGgxQinFqZhZsQTk7_cYZIk"
id_compradores_ssp = "1A3D8JNPlZNpxkwcXQ_7CxalNEZyLo8KmJP6Sa_A-3NU"

# === Carregar DataFrames ===
df_leads_trafego = carregar_aba(id_dados_leads_ssp, "Página1")
df_pesquisa_captacao = carregar_aba(id_pesquisa_ssp, "Página1")
df_compradores = carregar_aba(id_compradores_ssp, "Compradores")

TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018DAC8B0AA0>: Failed to resolve 'oauth2.googleapis.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
df_pesquisa_captacao = df_pesquisa_captacao.drop(["Funil"], axis=1, errors='ignore')
df_pesquisa_captacao.rename(columns={'E-mail': 'email'}, inplace=True)
df_pesquisa_captacao.columns = df_pesquisa_captacao.columns.str.lower()

df_pesquisa_captacao['data'] = pd.to_datetime(df_pesquisa_captacao['data'], errors='coerce')
df_pesquisa_captacao['data'] = df_pesquisa_captacao['data'].dt.tz_localize(None)
df_pesquisa_captacao['data'] = df_pesquisa_captacao['data'].dt.normalize()

df_pesquisa_captacao["email"] = df_pesquisa_captacao["email"].str.lower().str.strip()
df_pesquisa_captacao = df_pesquisa_captacao.drop_duplicates(subset='email', keep='first')

df_pesquisa_captacao.info()

In [ ]:
df_pesquisa_captacao["renda"] = (
    df_pesquisa_captacao["renda"]
    .replace({
        "O a R$1.000\u2028": "De 0 a 1.000",
        "R$1.000 a R$3.000": "De 1.000 a 3.000",
        "R$3.000 a R$5.000\u2028": "De 3.000 a 5.000",
        "R$5.000 a R$10.000\u2028": "De 5.000 a 10.000",
        "Mais de R$10.000": "Mais de 10.000"
    })
)

df_pesquisa_captacao["escolaridade"] = (
    df_pesquisa_captacao["escolaridade"]
    .replace({
        "Ensino superior incompleto\u2028": "Ensino superior incompleto"
    })
)

In [ ]:
# Filtrar o DataFrame para excluir linhas onde o email contém "teste" ou "x@x"
df_pesquisa_captacao = df_pesquisa_captacao[
    ~df_pesquisa_captacao['email'].str.contains('teste|x@x', case=False, na=False)
]

df_pesquisa_captacao.drop(columns=['email']).head()

In [ ]:
df_leads_trafego.info()

In [ ]:
df_leads_trafego.columns = df_leads_trafego.columns.str.strip().str.lower().str.replace(" ", "_")

# Definir colunas UTM
colunas_utms = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']

# Função para verificar se uma célula é "vazia de verdade"
def is_vazio(valor):
    if pd.isna(valor):
        return True
    valor = str(valor).strip()
    return (valor == "") or (valor.startswith("{{") and valor.endswith("}}"))

# Função para verificar se TODAS UTMs estão vazias para uma linha
def todas_utms_vazias(row):
    return all(is_vazio(row[col]) for col in colunas_utms)

# Aplicar filtro: manter linhas onde pelo menos uma UTM está preenchida de verdade
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego.apply(todas_utms_vazias, axis=1)
].copy()

# Remover linhas que contenham 'teste' em qualquer coluna
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego.apply(lambda row: row.astype(str).str.lower().str.contains('teste').any(), axis=1)
]

# Resetar índice
df_leads_trafego = df_leads_trafego.reset_index(drop=True)

df_leads_trafego.info()

In [ ]:
df_leads_trafego.rename(columns={'e-mail': 'email'}, inplace=True)
df_leads_trafego["email"] = df_leads_trafego["email"].str.lower().str.strip()

df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')
df_leads_trafego['data'] = df_leads_trafego['data'].dt.tz_localize(None)
df_leads_trafego['data'] = df_leads_trafego['data'].dt.normalize()

df_leads_trafego.info()

In [ ]:
# Criar coluna de lançamentos baseada na data
df_leads_trafego['lancamentos'] = df_leads_trafego['data'].apply(
    lambda x: 'SSP-L13' if x.date() >= pd.to_datetime('2025-04-30').date() else 'SSP-L12'
)

# Remover e-mails duplicados dentro de cada lançamento
df_leads_trafego = (
    df_leads_trafego
    .sort_values('data')  # garante ordem cronológica
    .drop_duplicates(subset=['email', 'lancamentos'], keep='first')
)

df_leads_trafego.info()

In [ ]:
df_leads_trafego["lancamentos"].value_counts()

In [ ]:
df_leads_trafego.drop(columns=['email']).head()

In [ ]:
df_compradores.rename(columns={'email contato': 'email'}, inplace=True)
df_compradores["email"] = df_compradores["email"].str.lower().str.strip()
df_compradores = df_compradores.drop_duplicates(subset='email', keep='first')

df_compradores.info()

In [ ]:
df_pesquisa_compradores_l12 = df_pesquisa_captacao[
    df_pesquisa_captacao["email"].isin(df_compradores["email"])
].copy()

In [ ]:
df_pesquisa_compradores_l12.info()

In [ ]:
df_pesquisa_compradores_l12.drop(columns=['email']).head()

In [ ]:
df_compradores_nao_respondentes = df_compradores[
    ~df_compradores["email"].isin(df_pesquisa_captacao["email"])
].copy()

In [ ]:
df_compradores_nao_respondentes.info()

In [ ]:
# Merge mantendo os lançamentos da pesquisa
df_pesquisa_captacao = df_pesquisa_captacao.merge(
    df_leads_trafego[['email'] + colunas_utms + ['lancamentos']],
    on='email',
    how='left'
)

df_pesquisa_compradores_l12 = df_pesquisa_compradores_l12.merge(
    df_leads_trafego[["email"] + colunas_utms + ["lancamentos"]],
    on="email",
    how="left"
)

In [ ]:
emails_pesquisa = set(df_pesquisa_captacao['email'].unique())
emails_trafego = set(df_leads_trafego['email'].unique())
emails_aluno = set(df_pesquisa_compradores_l12['email'].unique())

# Quantos e-mails da pesquisa não estão no tráfego?
len(emails_pesquisa - emails_trafego)

In [ ]:
df_pesquisa_captacao.drop(columns=['email']).head()

In [ ]:
df_pesquisa_compradores_l12.drop(columns=['email']).head()

In [ ]:
df_pesquisa_captacao.info()

In [ ]:
# Contar quantas linhas têm todas as UTMs como NaN
qtd_todas_utms_nan = df_pesquisa_compradores_l12[colunas_utms].isna().all(axis=1).sum()

print(f"Quantidade de linhas com TODAS as UTMs NaN: {qtd_todas_utms_nan}")

In [ ]:
"""
model = SentenceTransformer("all-MiniLM-L6-v2")

# Mapas distintos por coluna
mapeamentos = {
    'problema_aprender': mapeamento_problema_aprender,
    'profissao': mapeamento_profissoes,
    'fala_outro_idioma': mapeamento_outros_idiomas,
    'motivo_fluencia_espanhol': mapeamento_motivo_fluencia,
    'escolaridade': mapeamento_escolaridade
}

for coluna, mapeamento in mapeamentos.items():
    nome_coluna_categoria = f"{coluna}_categoria"

    # Expandir mapeamento para esta coluna
    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)

    # Categorização com o mapeamento certo
    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(
        df_pesquisa_captacao_lancamentos[coluna],
        mapeamento_expandido,
        mapeamento_embeddings,
        model,
        threshold=0.6,
        desc=coluna
    )
"""

In [ ]:
"""# Exibir strings completas
pd.set_option("display.max_colwidth", None)

# Colunas categorizadas e respectivas colunas originais
categorias = {
    "problema_aprender": "problema_aprender_categoria",
    "profissao": "profissao_categoria",
    "fala_outro_idioma": "fala_outro_idioma_categoria",
    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",
    "escolaridade": "escolaridade_categoria"
}

# Categorias problemáticas
categorias_problema = ["Outros"]

# Loop para filtrar e exibir como DataFrame formatado
for original_col, categoria_col in categorias.items():
    for problema in categorias_problema:
        print(f"\n=== Valores de '{original_col}' categorizados como '{problema}' ===")

        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema
        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()

        # Envolver cada valor com aspas
        valores.index = [f'"{val}"' for val in valores.index]

        # Converter para DataFrame para visualização completa
        valores_df = valores.reset_index().head(300)
        valores_df.columns = ['valor_original', 'frequencia']

        # Exibir o resultado completo
        print(valores_df.to_string(index=False))
"""

In [ ]:
"""df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([
    'profissao',
    'fala_outro_idioma',
    'motivo_fluencia_espanhol',
    'escolaridade',
    'problema_aprender'
], axis=1, errors='ignore')
"""

In [ ]:
from gspread_dataframe import set_with_dataframe
from datetime import datetime

# Carrega as variáveis de ambiente
load_dotenv()

# Pega o caminho de forma segura
credenciais_path = os.getenv("GOOGLE_CREDENTIALS_PATH")

# === Escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# === Autenticar ===
creds = service_account.Credentials.from_service_account_file(
    credenciais_path,
    scopes=scopes
)
client = gspread.authorize(creds)

# === Função para criar nova planilha e carregar dados ===
def criar_planilha_e_enviar(df, nome_base):
    nome_final = nome_base
    
    # Cria nova planilha
    planilha = client.create(nome_final)
    
    # Compartilha com seu e-mail pessoal (aqui você coloca o seu)
    planilha.share('camilobf2@gmail.com', perm_type='user', role='writer')  # <<< Trocar pelo seu email do Gmail

    # Preenche a primeira aba
    aba = planilha.sheet1
    aba.update_title("Dados")
    set_with_dataframe(aba, df)
    
    print(f"✅ Nova planilha criada: {nome_final}")
    print(f"🔗 Link: https://docs.google.com/spreadsheets/d/{planilha.id}/edit")

# === Geração dos arquivos ===
#criar_planilha_e_enviar(df_pesquisa_captacao, "pesquisa_captacao")
#criar_planilha_e_enviar(df_pesquisa_compradores_l12, "pesquisa_aluno")
#criar_planilha_e_enviar(df_leads_invest_trafego, "invest_trafego")

In [ ]:
# === Função para atualizar uma planilha existente ===
def atualizar_planilha_existente(df: pd.DataFrame, sheet_id: str, aba_nome: str = "Dados"):
    """
    Atualiza uma aba específica de uma planilha no Google Sheets.
    
    - df: DataFrame com os dados que serão enviados
    - sheet_id: ID da planilha (o que vem na URL depois de "/d/")
    - aba_nome: Nome da aba que será atualizada (default: 'Dados')
    """
    try:
        planilha = client.open_by_key(sheet_id)
        try:
            aba = planilha.worksheet(aba_nome)
        except gspread.WorksheetNotFound:
            aba = planilha.add_worksheet(title=aba_nome, rows="1000", cols="20")
        
        aba.clear()  # Limpa dados antigos
        set_with_dataframe(aba, df)
        
        print(f"✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/{sheet_id}/edit")
    except Exception as e:
        print(f"❌ Erro ao atualizar a planilha: {e}")

# === IDs das planilhas (FIXOS, preencha certinho aqui) ===
id_pesquisa_captacao = "1ukLwu8SoP0U3uirB6w1Ca3TPEITY50c558xNUUN3kj4"
id_pesquisa_aluno = "1GDCAa1fiflDIBnRY9rrdY9ghKwS6DPzxIG9FYGZEsQU"


# === Atualizar todas as planilhas ===
atualizar_planilha_existente(df_pesquisa_captacao, id_pesquisa_captacao)
atualizar_planilha_existente(df_pesquisa_compradores_l12, id_pesquisa_aluno)

In [ ]:
from datetime import datetime
import pytz

fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

with open("ultima_atualizacao.txt", "w") as f:
    f.write(agora_brasil.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
try:
    with open("ultima_atualizacao.txt", "r") as f:
        texto = f.read()
        data_atualizacao = datetime.strptime(texto, "%Y-%m-%d %H:%M:%S")
        data_atualizacao_formatada = data_atualizacao.strftime("%d/%m/%Y %H:%M")
except Exception:
    data_atualizacao_formatada = "Desconhecida"

In [ ]:
f"**Última atualização:** {data_atualizacao_formatada}"

In [ ]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_captacao.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_captacao[coluna].value_counts(dropna=False))

In [ ]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_compradores_l12.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_compradores_l12[coluna].value_counts(dropna=False))